In [2]:
import os
from pyciemss.Ensemble.interfaces import (
    load_and_sample_petri_ensemble, load_and_calibrate_and_sample_ensemble_model
)
from pyciemss.visuals import plots

In [4]:
DEMO_PATH = "../../notebook/integration_demo/"
# ASKENET_PATH_1 = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json"
ASKENET_PATH_1 = "../../test/models/AMR_examples/ensemble/SEIARHDS_AMR.json"
ASKENET_PATH_2 = "../../test/models/AMR_examples/ensemble/SIRHD_AMR.json"

ASKENET_PATHS = [ASKENET_PATH_1, ASKENET_PATH_2]

## load_and_sample_ensemble_model

In [3]:
weights = [0.5, 0.5]
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]
solution_mappings = [{"Infected": "Cases", "Hospitalizations": "hospitalized_population"}, # model 1 mappings
                     {"Infected": "Infections", "Hospitalizations": "hospitalized_population"} # model 2
                     ]

# Run sampling
result = load_and_sample_petri_ensemble(
    ASKENET_PATHS, weights, solution_mappings, num_samples, timepoints, 
    time_unit="days",
    visual_options={"keep":".*_sol"}
)

ensemble_samples = result["data"]
q_ensemble = result["quantiles"]

# Save results
ensemble_samples.to_csv(os.path.join(DEMO_PATH, "results_petri_ensemble/sample_results.csv"), index=False)
q_ensemble.to_csv(os.path.join(DEMO_PATH, "results_petri_ensemble/quantile_results.csv"), index=False)

In [4]:
plots.ipy_display(result["visual"])

## load_and_calibrate_and_sample_ensemble_model

In [5]:
# # Generate data from model
# from pyciemss.utils.interface_utils import solutions_to_observations

# data = solutions_to_observations(timepoints, result["data"].set_index(["timepoint_id", "sample_id"]))
# data[0].to_csv(os.path.join(DEMO_PATH, "results_petri_ensemble/ensemble_data.csv"), index=False)

In [6]:
data_path = os.path.join(DEMO_PATH, "results_petri_ensemble/ensemble_data.csv")
num_samples = 2
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]

# Run the calibration and sampling
result = load_and_calibrate_and_sample_ensemble_model(
    ASKENET_PATHS,
    data_path,
    weights,
    solution_mappings,
    num_samples,
    timepoints,
    verbose=True,
    total_population=1000,
    num_iterations=10,
    time_unit="days",
    visual_options={"title": "Calibrated Ensemble", "keep":".*_sol"}
)

# Save results
result["data"].to_csv(
    os.path.join(DEMO_PATH, "results_petri_ensemble/calibrated_sample_results.csv"), index=False
)
result["quantiles"].to_csv(
    os.path.join(DEMO_PATH, "results_petri_ensemble/calibrated_quantile_results.csv"), index=False
)
plots.ipy_display(result["visual"])

iteration 0: loss = 21.878735542297363



In [5]:
from pyciemss.utils.interface_utils import cdc_reformatcsv

q_ensemble_data = cdc_reformatcsv(filename=os.path.join(DEMO_PATH, "results_petri_ensemble/calibrated_quantile_results.csv"), 
                                  solution_string_mapping={"Infected": "inc case", "Hospitalizations": "inc hosp.", "D": "cum death"}, 
                                  forecast_start_date="2023-08-03",
                                  location="US",
                                  drop_column_names=["timepoint_id", "number_days", "inc_cum", "output", "Forecast_Backcast"])
q_ensemble_data.to_csv(os.path.join(DEMO_PATH, "results_petri_ensemble/CDCformat_calibrated_quantile_results.csv"), index=False)